## Imports

In [54]:
%load_ext autoreload
%autoreload 2
%load_ext nb_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


<IPython.core.display.Javascript object>

In [55]:
from pathlib import Path
from random import choice
from PIL import Image, ImageDraw, ImageEnhance
from random import random
import numpy as np
import sys
sys.path.append("..")
from src.preprocessing import crop_transparent, paste_img_on_bg, init_coco_data
import json
from datetime import datetime

<IPython.core.display.Javascript object>

## Prepare rendered images

* Crop to non-transparent area

In [56]:
object_paths = Path("../data/rendered/mbp")
object_paths = [x for x in object_paths.iterdir()]

<IPython.core.display.Javascript object>

In [57]:
for object_path in object_paths:
    img = Image.open(object_path)
    img = crop_transparent(img)
    filename = object_path.parts[-1]
    img.save(f"../data/processed/cropped_render/mbp/{filename}.png")

<IPython.core.display.Javascript object>

In [58]:
bgs = Path("../data/raw/flat_bg/")
bgs = [x for x in bgs.iterdir()]

<IPython.core.display.Javascript object>

In [59]:
cropped_object_paths = Path("../data/processed/cropped_render/mbp")
cropped_object_paths = [x for x in cropped_object_paths.iterdir()]

<IPython.core.display.Javascript object>

In [60]:
data = init_coco_data()
data["info"] = {
    "description": "Synthetic test dataset",
    "version": "1.0",
    "year": 2021,
    "contributor": "Poplar Studio",
    "date_created": datetime.now().strftime("%Y-%m-%d")
}
mbp_category = {
    "id": 1,
    "name": "mbp"
}
data["categories"].append(mbp_category)

for i, object_path in enumerate(cropped_object_paths):
    background = Image.open(choice(bgs))
    object_img = Image.open(object_path)
    result, bbox = paste_img_on_bg(object_img, background)
    file_name = f"mbp_{i}.png"
    result.save(f"../data/processed/training/mbp/{file_name}")
    
    image_data = {
        "license": 1,
        "file_name": file_name,
        "height": result.size[1],
        "width": result.size[0],
        "id": i
    }
    
    category_id = 0
    annotation_data = {
        "image_id": i,
        "bbox": bbox,
        "category_id": 1,
        "id": i,
    }
    
    data["images"].append(image_data)
    data["annotations"].append(annotation_data)

<IPython.core.display.Javascript object>

In [61]:
data

{'info': {'description': 'Synthetic test dataset',
  'version': '1.0',
  'year': 2021,
  'contributor': 'Poplar Studio',
  'date_created': '2021-09-17'},
 'licenses': [{'url': 'http://creativecommons.org/licenses/by-nc-sa/2.0/',
   'id': 1,
   'name': 'Attribution-NonCommercial-ShareAlike License'}],
 'images': [{'license': 1,
   'file_name': 'mbp_0.png',
   'height': 1440,
   'width': 1920,
   'id': 0},
  {'license': 1,
   'file_name': 'mbp_1.png',
   'height': 1440,
   'width': 1920,
   'id': 1},
  {'license': 1,
   'file_name': 'mbp_2.png',
   'height': 1440,
   'width': 1920,
   'id': 2},
  {'license': 1,
   'file_name': 'mbp_3.png',
   'height': 1440,
   'width': 1920,
   'id': 3},
  {'license': 1,
   'file_name': 'mbp_4.png',
   'height': 1440,
   'width': 1920,
   'id': 4}],
 'annotations': [{'image_id': 0,
   'bbox': [458, 94, 994, 1101],
   'category_id': 1,
   'id': 0},
  {'image_id': 1, 'bbox': [719, 711, 738, 423], 'category_id': 1, 'id': 1},
  {'image_id': 2, 'bbox': [666,

<IPython.core.display.Javascript object>

In [62]:
with open("../data/processed/training/mbp/data.json", "w") as fp:
    json.dump(data, fp)


<IPython.core.display.Javascript object>